In [ ]:
import numpy as np
import cv2 as cv
import os
import json
import matplotlib.pyplot as plt
import apriltag
from scipy.spatial.transform import Rotation

import lac.params as params
from lac.utils.frames import apply_transform

%load_ext autoreload
%autoreload 2

# Loading fiducial locations


In [ ]:
# known fiducial corner positions
fiducial_group_centers = json.load(
    open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json"))
)["lander"]["fiducials"]
centers_group_a = fiducial_group_centers["a"]
centers_group_b = fiducial_group_centers["b"]
centers_group_c = fiducial_group_centers["c"]
centers_group_d = fiducial_group_centers["d"]

FIDUCIAL_NAMES = ["top left", "top right", "lower right", "lower left"]


def fiducial_centers_array(centers: dict) -> np.ndarray:
    center_pts = []
    for name in FIDUCIAL_NAMES:
        center_pts.append(np.array([centers[name]["x"], centers[name]["y"], centers[name]["z"]]))
    return np.array(center_pts)

In [ ]:
# These angles are listed clockwise (like bruh)
tag_group_bearing_angles = {
    "a": 135,
    "b": 45,
    "c": 315,
    "d": 225,
}

In [ ]:
tag_poses = {}

for group, group_vals in fiducial_group_centers.items():
    for tag, tag_vals in group_vals.items():
        tag_poses[tag_vals["id"]] = {
            "center": np.array([tag_vals["x"], tag_vals["y"], tag_vals["z"]]),
            "bearing": tag_group_bearing_angles[group],
        }

Test getting tag corners in world frame for an example tag


In [ ]:
tag_id = 37  # (Group A, lower right)
tag_center = tag_poses[tag_id]["center"]
tag_bearing = tag_poses[tag_id]["bearing"]

In [ ]:
half_size = params.TAG_SIZE / 2
tag_corners_local = np.array(
    [
        [half_size, 0, half_size],
        [-half_size, 0, half_size],
        [-half_size, 0, -half_size],
        [half_size, 0, -half_size],
    ]
)
R = Rotation.from_euler("z", -tag_bearing, degrees=True).as_matrix()
tag_corners_lander = tag_corners_local @ R.T + tag_center
print(tag_corners_lander)

In [ ]:
Rotation.from_euler("z", -tag_bearing, degrees=True).apply(tag_corners_local) + tag_center

In [ ]:
data_path = "../../results/runs/map1_seed4_spiral_4.5_2.0_run2/data_log.json"
json_data = json.load(open(f"{data_path}"))
lander_pose = np.array(json_data["lander_pose_world"])

tag_corners_world = apply_transform(lander_pose, tag_corners_lander)
print(tag_corners_world)

# AprilTag detection


In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/lander_closeups/")
CAM_NAME = "FrontLeft"
imgs = os.listdir(os.path.join(data_path, CAM_NAME))
imgs = [os.path.join(data_path, CAM_NAME, imgs[i]) for i in range(0, len(imgs), 1)]

In [ ]:
i = 1165
img = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)
plt.imshow(img, cmap="gray")

In [ ]:
options = apriltag.DetectorOptions(families="tag36h11")
detector = apriltag.Detector(options)
results = detector.detect(img)

In [ ]:
results

In [ ]:
results[0].corners

From testing once this detector seems worse (and also only gives integer pixel values)


In [ ]:
import cv2.aruco as aruco

# Define AprilTag dictionary
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_APRILTAG_36h11)
parameters = aruco.DetectorParameters()
detector = aruco.ArucoDetector(aruco_dict, parameters)

# Detect markers
corners, ids, _ = detector.detectMarkers(img)

# Solving for camera pose using PnP


In [ ]:
from lac.perception.pnp import get_tag_corners_world

In [ ]:
success, rvec, tvec = cv.solvePnP(
    objectPoints=get_tag_corners_world(results[0].tag_id, lander_pose),
    imagePoints=results[0].corners,
    cameraMatrix=params.CAMERA_INTRINSICS,
    distCoeffs=None,
    flags=cv.SOLVEPNP_ITERATIVE,
)
if success:
    print("Rotation Vector (rvec):", rvec)
    print("Translation Vector (tvec):", tvec)
    # Convert rotation vector to rotation matrix
    R, _ = cv.Rodrigues(rvec)
    # Compute homogeneous transformation matrix
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = tvec.flatten()
    print("Homogeneous Transformation Matrix:\n", T)
    # Compute distance
    distance = np.linalg.norm(tvec)  # Euclidean distance
    print(f"Distance to AprilTag: {distance:.2f} meters")

In [ ]:
Rotation.from_matrix(R).as_euler("xyz", degrees=True)

In [ ]:
from lac.utils.frames import invert_transform_mat


T_inv = invert_transform_mat(T)
print(np.round(T_inv, 2))
print(Rotation.from_matrix(T_inv[:3, :3]).as_euler("xyz", degrees=True))